# BAX 422 Midterm

In [7]:
# importing libraries 
import pandas as pd
import numpy as np
import bs4 as bs
import requests
import time
import re
import os
import json
import pymysql
import mysql.connector
import pprint
from sqlalchemy import create_engine

## Part 1

#### a.

In [2]:
# Accessing website content using API and parsing JSON string to Python objects
url = "http://numbersapi.com/1..99"
response = requests.get(url)
trivia = json.loads(str(response.text))

In [5]:
# Creating fact indices
str_num1to9 = ["{:1d}".format(n) for n in np.array(range(1,10))]
str_num10to99 = ["{:1d}".format(n) for n in np.array(range(10,100))]

In [6]:
# Looping and printing trivia
try:
    for n in str_num1to9:
        print('00'+n, '-',trivia[n])

    for n in str_num10to99:
        print('0'+n, '-',trivia[n])
except IOError as e:
    print(e)

001 - 1 is the loneliest number.
002 - 2 is the number of polynucleotide strands in a DNA double helix.
003 - 3 is the number of words or phrases in a Tripartite motto.
004 - 4 is the number of legs most furniture has.
005 - 5 is the most common number of gears for automobiles with manual transmission.
006 - 6 is the number of fundamental flight instruments lumped together on a cockpit display.
007 - 7 is the number of colors of the rainbow.
008 - 8 is the number of furlongs in a mile.
009 - 9 is the number of innings in a regulation, non-tied game of baseball.
010 - 10 is the number of hydrogen atoms in butane, a hydrocarbon.
011 - 11 is the miles per hours that the fastest moving land snake, the Black Mamba, can move.
012 - 12 is the largest number that has just one syllable.
013 - 13 is the number of loaves in a "baker's dozen".
014 - 14 is the number of pieces the body of Osiris was torn into by his fratricidal brother Set.
015 - 15 is the number of letters in the words "uncopyrigh

## Part 2

#### a. 

In [5]:
# You would add "&rt=nc&LH_BIN=1" at the end of the url
# "rt=" is the variable that corresponding to buy-it-now searches
# You would add "&_ipg=100" at the end of the of the url
# "_ipg=" is the variable name corresponding to items per page searches

#### b

In [6]:
# Setting browser 
header = {'User-agent': 'Mozilla/5.0'} 

In [7]:
# Setting page indices
page = ["{:1d}".format(n) for n in np.array(range(10))+1]
page

['1', '2', '3', '4', '5', '6', '7', '8', '9', '10']

In [8]:
# Getting sponsored and non-sponsored links
try:

    links_sponsored = []
    links_nonsponsored = []

    # Looping through the pages
    for p in page:
    
        # Sleepy time zzz
        # time.sleep(2)
    
        # Acessing website and making soup
        url = "https://www.ebay.com/sch/i.html?_from=R40&_nkw=canon+eos+5d&_sacat=0&_ipg=100&rt=nc&LH_BIN=1&_pgn="+p
        response = requests.get(url, headers=header)
        soup = bs.BeautifulSoup(response.text,'html.parser')
    
        # Fetching relevent tags in source sode
        items = soup.select("#srp-river-results > ul > li.s-item")
        for item in items:
            a = item.find("a")
            if not a:
                continue
        
            # Storing sponsored and non-sponsored urls in relevant lists       
            text = item.find("span").get_text()
            if re.search('.*S.*P.*O.*N.*S.*O.*R.*E.*D.*',text): 
                link = {}
                link = re.sub("(.*)\\?.*", r"\1", a['href'])
                links_sponsored.append(link)
            else:
                link = {}
                link = re.sub("(.*)\\?.*", r"\1", a['href'])
                links_nonsponsored.append(link)
                
except IOError as e:
    print(e)

In [9]:
# Writing sponsored links to txt file
with open('sponsored.txt', 'w') as sponsored:
    for link in links_sponsored:
        sponsored.write('%s\n' % link)

In [10]:
# Writing non-sponsored links to txt file
with open('non-sponsored.txt', 'w') as non_sponsored:
    for link in links_nonsponsored:
        non_sponsored.write('%s\n' % link)

#### c

In [11]:
# Creating folders for sponsored and non-sponsored 
def createFolder(folder):
    try:
        if not os.path.exists(folder):
            os.makedirs(folder)
    except OSError:
        print ('Error: Creating directory. ' +  folder)
        
createFolder('./sponsored/')
createFolder('./non-sponsored/')

In [8]:
# Saving current directory 
current_wkd = os.getcwd()

In [9]:
current_wkd

'/Users/nandinibasu/Desktop/MyDocs/MSBA/Winter /BAX 422/Midterm'

In [13]:
# Creating a list the size of sponsored products to access all products in sponsored.txt file
length_s = len(links_sponsored)
link_range_s = list(range(0, length_s))

In [14]:
# using sub-string to get ID for sponsored 
sponsored_files = []
try:
    for link in link_range_s:
        
        # Sleepy time zzz
        # time.sleep(2)
        
        # Opening each product url
        product_url = links_sponsored[link]
        page_response = requests.get(product_url, headers=header)
        
        # Fetching product id and naming new file
        id = product_url[-12:]
        pagename = id +".htm"
        
        # Opening folder by setting new directory to write product htm file in 
        os.chdir(current_wkd+'/sponsored')
        f = open(pagename, 'w')
        f.write(page_response.text)
        f.close()
        
        sponsored_files.append(pagename)
    
    # Re-setting working directory 
    os.chdir(current_wkd)
    
except Exception as e:
    print(e)

In [15]:
# Creating a list the size of non-sponsored products to access all products in non-sponsored.txt file
length_ns = len(links_nonsponsored)
link_range_ns = list(range(0, length_ns))

In [16]:
# using sub-string to get ID for non-sponsored
nonsponsored_files = []

try:
    for link in link_range_ns:
        
        # Sleepy time zzz
        # time.sleep(2)
                
        # Opening each product url
        product_url = links_nonsponsored[link]
        page_response = requests.get(product_url, headers=header)
        
        # Fetching product id and naming new file
        id = product_url[-12:]
        pagename = id +".htm"
    
        # Opening folder by setting new directory to write product htm file in 
        os.chdir(current_wkd+'/non-sponsored')
        f = open(pagename, 'w')
        f.write(page_response.text)
        f.close()

        nonsponsored_files.append(pagename)
        
    # Re-setting working directory 
    os.chdir(current_wkd)

except Exception as e:
    print(e)

('Connection aborted.', OSError("(60, 'ETIMEDOUT')"))


#### d

In [17]:
# Creating a list the size of sponsored products to access all products in sponsored folder
length_files_spon = len(sponsored_files)
spon_files_range = list(range(0, length_files_spon))

In [18]:
# Pulling all information for sponsored products
try:
    spon_contents = []
    ret = []
    for file in spon_files_range:
        # opening sponsored folder 
        os.chdir(current_wkd+'/sponsored')
        
        # Opening each product from folder
        f = open(sponsored_files[file], 'r')
        page = f.read()
        f.close()
        # making soup 
        sponsored_soup = bs.BeautifulSoup(page,'html.parser')
        content = {}
    
        # Getting name using selectors h1 and id:itemTitle
        n = sponsored_soup.find("h1", {"id": "itemTitle"}).text
        content['item_name'] = n[16:]
    
        # Getting price using selectors span and class:notranslate
        price = sponsored_soup.find("span", {"class": "notranslate"}).text.strip('\n').strip('\t')
        if 'US' in price:
            price = price[2:]
            content['price'] = price 
            content['currency'] = "US"
        else:
            content['price'] = price
            content['currency'] = 'CA'
        
        # Getting condition of item using selectors div and class:u-flL condText
        content['item_condition'] = sponsored_soup.find("div", {"class": "u-flL condText"}).text
    
        # Getting information on the seller using selectors div and class:si-cnt si-cnt-eu vi-grBr vi-padn0 c-std
        seller_section = sponsored_soup.find("div", {"class": "si-cnt si-cnt-eu vi-grBr vi-padn0 c-std"})
        # Getting seller name using selectors span and class:mbg-nw
        content['seller_name'] = seller_section.find("span", {"class": "mbg-nw"}).text
        # Getting seller score using selectors span and class:mbg-l and then .a to get to child
        content['seller_score'] = seller_section.find("span", {"class": "mbg-l"}).a.text
    
        # Getting availablity of best offer using selectors a and id:boBtn_btn
        best_offer = sponsored_soup.find("a", {"id": "boBtn_btn"})
        if best_offer is not None:
            content['best_offer'] = "Available"
        else:
            content['best_offer'] = "Not Available"
    
        # Getting number of items sold using selectors a and class:vi-txt-underline
        no_sold = sponsored_soup.find("a", {"class": "vi-txt-underline"})
        if no_sold is not None:
            content['no_sold'] = no_sold.text[:-4]
        else:
            content['no_sold'] = 0
    
        # Getting return information using selectors span and class:w2b-sgl then searching for returns in list
        for t in sponsored_soup.find_all("span", {"class": "w2b-sgl"}):
            r = {}
            r = t.text
            ret.append(r)
            if 'No returns' in ret:
                content['returns_allowed'] = 0
            else:
                content['returns_allowed'] = 1
            ret = []
            
        # Getting shipping price using selectors span and id:shSummary and then .span to get to child
        shipping = sponsored_soup.find("span", {"id": "shSummary"}).span.text.strip('\n').strip('\t')
        if shipping == 'FREE':
            content["shipping"] = "Free"
        elif shipping == "FAST 'N FREE":
            content["shipping"] = "Free"
        elif shipping == 'Varies based on location and shipping method':
            content["shipping"] = "Calculated"
        elif shipping == 'Local pick-up offered.':
            content["shipping"] = "Pick up"
        else:
            content["shipping"] = shipping
        
        # Adding everything to final list    
        spon_contents.append(content)
        # Re-setting working directory
        os.chdir(current_wkd)
        
except Exception as e:
    print(e)

In [19]:
# Creating a list the size of non-sponsored products to access all products in non-sponsored folder
length_files_nonspon = len(nonsponsored_files)
nonspon_files_range = list(range(0, length_files_nonspon))

In [20]:
# Pulling all information for non-sponsored products
try:
    nonspon_contents = []
    ret = []
    for file in nonspon_files_range:
        # opening sponsored folder 
        os.chdir(current_wkd+'/non-sponsored')
    
        # Opening each product from folder
        f = open(nonsponsored_files[file], 'r')
        page = f.read()
        f.close()
        # making soup 
        nonsponsored_soup = bs.BeautifulSoup(page,'html.parser')
        content = {}
    
        # Getting name using selectors h1 and id:itemTitle
        n = nonsponsored_soup.find("h1", {"id": "itemTitle"}).text
        content['item_name'] = n[16:]
    
        # Getting price using selectors span and class:notranslate
        price = nonsponsored_soup.find("span", {"class": "notranslate"}).text.strip('\n').strip('\t')
        if 'US' in price:
            price = price[2:]
            content['price'] = price 
            content['currency'] = "US"
        else:
            content['price'] = price
            content['currency'] = 'CA'
    
        # Getting condition of item using selectors div and class:u-flL condText
        content['item_condition'] = nonsponsored_soup.find("div", {"class": "u-flL condText"}).text
    
        # Getting information on the seller using selectors div and class:si-cnt si-cnt-eu vi-grBr vi-padn0 c-std
        seller_section = nonsponsored_soup.find("div", {"class": "si-cnt si-cnt-eu vi-grBr vi-padn0 c-std"})
        # Getting seller name using selectors span and class:mbg-nw
        content['seller_name'] = seller_section.find("span", {"class": "mbg-nw"}).text
        # Getting seller score using selectors span and class:mbg-l and then .a to get to child
        content['seller_score'] = seller_section.find("span", {"class": "mbg-l"}).a.text
    
        # Getting availablity of best offer using selectors a and id:boBtn_btn
        best_offer = nonsponsored_soup.find("a", {"id": "boBtn_btn"})
        if best_offer is not None:
            content['best_offer'] = "Available"
        else:
            content['best_offer'] = "Not Available"
    
        # Getting number of items sold using selectors a and class:vi-txt-underline
        no_sold = nonsponsored_soup.find("a", {"class": "vi-txt-underline"})
        if no_sold is not None:
            content['no_sold'] = no_sold.text[:-4]
        else:
            content['no_sold'] = 0
    
        # Getting return information using selectors span and class:w2b-sgl then searching for returns in list
        for t in nonsponsored_soup.find_all("span", {"class": "w2b-sgl"}):
            r = {}
            r = t.text
            ret.append(r)
            if 'No returns' in ret:
                content['returns_allowed'] = 0
            else:
                content['returns_allowed'] = 1
            ret = []
            
        # Getting shipping price using selectors span and id:shSummary and then .span to get to child
        shipping = nonsponsored_soup.find("span", {"id": "shSummary"}).span.text.strip('\n').strip('\t')
        if shipping == 'FREE':
            content["shipping"] = "Free"
        elif shipping == "FAST 'N FREE":
            content["shipping"] = "Free"
        elif shipping == 'Varies based on location and shipping method':
            content["shipping"] = "Calculated"
        elif shipping == 'Local pick-up offered.':
            content["shipping"] = "Pick up"
        else:
            content["shipping"] = shipping
        
        # Adding everything to final list    
        nonspon_contents.append(content)
        # Re-setting working directory
        os.chdir(current_wkd)
        
except Exception as e:
    print(e)

In [21]:
# Converting lists to dataframes
spon_data = pd.DataFrame(spon_contents)
nonspon_data = pd.DataFrame(nonspon_contents)

In [22]:
# Adding sponsored and non-sponsored types as columns
spon_data['type'] = "Sponsored"
nonspon_data['type'] = "Non-sponsored"

In [23]:
# Data check
spon_data.head()

,item_name,price,currency,item_condition,seller_name,seller_score,best_offer,no_sold,returns_allowed,shipping,type
0,Canon EOS 5D Mark II 21.1MP Digital Camera Bod...,$448.00,US,Used,matsuiyastore,21169,Not Available,0,1,Free,Sponsored
1,Canon EOS 5D Mark III 22.3MP Digital SLR Camer...,"$1,250.00",US,Used,blinaldi14,9,Available,0,1,Calculated,Sponsored
2,Canon EOS 5D Digital SLR Camera Body {12.8 M/P},$339.00,US,Seller refurbished,kehoutlet,99376,Not Available,21,1,Free,Sponsored
3,Canon EOS 5D Mark II 21.1MP Digital SLR Camera...,$415.00,US,Used,rottnapples_closet,2,Available,0,0,Calculated,Sponsored
4,Canon EOS 5D Mark I 12.8MP Digital SLR Camera ...,$299.99,US,Used,otaku808,117,Available,0,1,Calculated,Sponsored


In [24]:
# Data check
nonspon_data.head(5)

,item_name,price,currency,item_condition,seller_name,seller_score,best_offer,no_sold,returns_allowed,shipping,type
0,Canon EOS 5D Mark IV / MK 4 DSLR Camera (Body ...,"$3,299.00",CA,New,deals-all-year,27942,Not Available,"2,379",1.0,Free,Non-sponsored
1,Canon EOS 5D Mark II 21.1MP Digital SLR Camera...,$379.00,US,Used,dan_812,8033,Not Available,0,1.0,Free,Non-sponsored
2,Canon EOS 5D Mark III Camera - Black (Body Onl...,$800.00,US,Used,lensauthority,224,Available,1,1.0,Free,Non-sponsored
3,Canon EOS 5D Mark II 21.1MP Digital SLR Camera...,$274.00,US,Used,waynelee748,319,Not Available,0,1.0,Calculated,Non-sponsored
4,Canon EOS 5DSR / 5DS R / 5D SR Digital SLR DSL...,"$3,699.00",CA,New,deals-all-year,27942,Not Available,748,1.0,Free,Non-sponsored


#### d

In [25]:
# Moving decimal places in price for sponsored
spon_data['price'] = spon_data['price'].str.replace(r'\D', '')

# Moving decimal places in shipping price for sponsored

for x in range(0,len(spon_data)):
    if spon_data['shipping'][x] == 'Free':
        spon_data['shipping'][x] = '0'
    elif spon_data['shipping'][x] == 'Calculated':
        spon_data['shipping'][x] = '0'
    elif spon_data['shipping'][x] == 'Pick up':
        spon_data['shipping'][x] = '0'
    else: 
        continue
    
spon_data['shipping'] = spon_data['shipping'].str.replace(r'\D', '')

# Removing comma and filling NAs
spon_data['no_sold'] = spon_data['no_sold'].str.replace(',', '')
spon_data['no_sold'] = spon_data['no_sold'].fillna(0)

/Users/nandinibasu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/nandinibasu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/nandinibasu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

In [26]:
# converting to int
spon_data[["price", "shipping", "no_sold", "seller_score"]] = spon_data[["price", "shipping", "no_sold", "seller_score"]].apply(pd.to_numeric)

In [27]:
# Data check
spon_data.head()

,item_name,price,currency,item_condition,seller_name,seller_score,best_offer,no_sold,returns_allowed,shipping,type
0,Canon EOS 5D Mark II 21.1MP Digital Camera Bod...,44800,US,Used,matsuiyastore,21169,Not Available,0,1,0,Sponsored
1,Canon EOS 5D Mark III 22.3MP Digital SLR Camer...,125000,US,Used,blinaldi14,9,Available,0,1,0,Sponsored
2,Canon EOS 5D Digital SLR Camera Body {12.8 M/P},33900,US,Seller refurbished,kehoutlet,99376,Not Available,21,1,0,Sponsored
3,Canon EOS 5D Mark II 21.1MP Digital SLR Camera...,41500,US,Used,rottnapples_closet,2,Available,0,0,0,Sponsored
4,Canon EOS 5D Mark I 12.8MP Digital SLR Camera ...,29999,US,Used,otaku808,117,Available,0,1,0,Sponsored


In [28]:
# Moving decimal places in price for non-sponsored
nonspon_data['price'] = nonspon_data['price'].str.replace(r'\D', '')

# Moving decimal places in Shipping price for non-sponsored

for x in range(0,len(nonspon_data)):
    if nonspon_data['shipping'][x] == 'Free':
        nonspon_data['shipping'][x] = '0'
    elif nonspon_data['shipping'][x] == 'Calculated':
        nonspon_data['shipping'][x] = '0'
    elif nonspon_data['shipping'][x] == 'Pick up':
        nonspon_data['shipping'][x] = '0'
    else: 
        continue
    
nonspon_data['shipping'] = nonspon_data['shipping'].str.replace(r'\D','')

# Removing comma and filling NAs
nonspon_data['no_sold'] = nonspon_data['no_sold'].str.replace(',', '')
nonspon_data['no_sold'] = nonspon_data['no_sold'].fillna(0)

/Users/nandinibasu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/Users/nandinibasu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/Users/nandinibasu/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-

In [29]:
# converting to int
nonspon_data[["price", "shipping", "no_sold", "seller_score"]] = nonspon_data[["price", "shipping", "no_sold", "seller_score"]].apply(pd.to_numeric)

In [30]:
# Data check
nonspon_data.head()

,item_name,price,currency,item_condition,seller_name,seller_score,best_offer,no_sold,returns_allowed,shipping,type
0,Canon EOS 5D Mark IV / MK 4 DSLR Camera (Body ...,329900,CA,New,deals-all-year,27942,Not Available,2379,1.0,0.0,Non-sponsored
1,Canon EOS 5D Mark II 21.1MP Digital SLR Camera...,37900,US,Used,dan_812,8033,Not Available,0,1.0,0.0,Non-sponsored
2,Canon EOS 5D Mark III Camera - Black (Body Onl...,80000,US,Used,lensauthority,224,Available,1,1.0,0.0,Non-sponsored
3,Canon EOS 5D Mark II 21.1MP Digital SLR Camera...,27400,US,Used,waynelee748,319,Not Available,0,1.0,0.0,Non-sponsored
4,Canon EOS 5DSR / 5DS R / 5D SR Digital SLR DSL...,369900,CA,New,deals-all-year,27942,Not Available,748,1.0,0.0,Non-sponsored


In [31]:
# combining sponsored and non-sponsored into one data frame
ebay_data = spon_data.append(nonspon_data)

In [32]:
# final check
ebay_data.head()

,item_name,price,currency,item_condition,seller_name,seller_score,best_offer,no_sold,returns_allowed,shipping,type
0,Canon EOS 5D Mark II 21.1MP Digital Camera Bod...,44800,US,Used,matsuiyastore,21169,Not Available,0,1.0,0.0,Sponsored
1,Canon EOS 5D Mark III 22.3MP Digital SLR Camer...,125000,US,Used,blinaldi14,9,Available,0,1.0,0.0,Sponsored
2,Canon EOS 5D Digital SLR Camera Body {12.8 M/P},33900,US,Seller refurbished,kehoutlet,99376,Not Available,21,1.0,0.0,Sponsored
3,Canon EOS 5D Mark II 21.1MP Digital SLR Camera...,41500,US,Used,rottnapples_closet,2,Available,0,0.0,0.0,Sponsored
4,Canon EOS 5D Mark I 12.8MP Digital SLR Camera ...,29999,US,Used,otaku808,117,Available,0,1.0,0.0,Sponsored


#### f

In [34]:
# Connecting to MySQL
conn = pymysql.connect(host='localhost', user='root')
cursor = conn.cursor()
mydb = conn.get_server_info()
print(mydb)

8.0.17


In [35]:
# Creating database ebay
query = "CREATE DATABASE IF NOT EXISTS ebay ;"
print(query)
cursor.execute(query)
conn.commit()

CREATE DATABASE IF NOT EXISTS ebay ;


/Users/nandinibasu/opt/anaconda3/lib/python3.7/site-packages/pymysql/cursors.py:170: Warning: (1007, "Can't create database 'ebay'; database exists")
  result = self._query(query)


In [36]:
# Creating table
query = "CREATE TABLE IF NOT EXISTS ebay.ebay_items (item_name VARCHAR(100) NOT NULL PRIMARY KEY, price INT(50), currency CHAR(10), item_condition VARCHAR(20), seller_name VARCHAR(100), seller_score VARCHAR(20), best_offer VARCHAR(20), no_sold VARCHAR(20), return_allowed VARCHAR(20), shipping INT(10), type VARCHAR(20));"
print(query)
cursor.execute(query)
conn.commit


CREATE TABLE IF NOT EXISTS ebay.ebay_items (item_name VARCHAR(100) NOT NULL PRIMARY KEY, price INT(50), currency CHAR(10), item_condition VARCHAR(20), seller_name VARCHAR(100), seller_score VARCHAR(20), best_offer VARCHAR(20), no_sold VARCHAR(20), return_allowed VARCHAR(20), shipping INT(10), type VARCHAR(20));


/Users/nandinibasu/opt/anaconda3/lib/python3.7/site-packages/pymysql/cursors.py:170: Warning: (1681, 'Integer display width is deprecated and will be removed in a future release.')
  result = self._query(query)
/Users/nandinibasu/opt/anaconda3/lib/python3.7/site-packages/pymysql/cursors.py:170: Warning: (1050, "Table 'ebay_items' already exists")
  result = self._query(query)


<bound method Connection.commit of <pymysql.connections.Connection object at 0x118c82550>>

In [37]:
# Establishing connection to mysql
engine = create_engine("mysql+pymysql://{user}@localhost/{db}".format(user="root", db="ebay"))

# inserting ebay_items into table
ebay_data.to_sql(name = 'ebay_items', con=engine, if_exists='replace', index=False)
conn.commit()

In [38]:
# Printing stats on price 
cursor.execute("select type, item_condition, max(price), min(price), avg(price), std(price) from ebay.ebay_items group by type, item_condition;")
result = cursor.fetchall()
price_stats =[]
for i in result:
    p ={}
    p = i
    price_stats.append(p)

In [39]:
price_stats = pd.DataFrame(price_stats)
price_stats.columns =['Type', 'Condition', 'Max', 'Min', 'Avg', 'SD'] 
price_stats

,Type,Condition,Max,Min,Avg,SD
0,Sponsored,Used,384600,22900,131249.5556,101222.026076
1,Sponsored,Seller refurbished,300000,33900,166950.0000,133050.000000
2,Sponsored,New,563799,48000,365720.3099,90673.053440
3,Sponsored,Open box,300000,299999,299999.5000,0.500000
4,Non-sponsored,New,449999,120000,314563.1429,97532.883626
5,Non-sponsored,Used,298900,15000,72543.9741,48775.283273
6,Non-sponsored,Seller refurbished,309900,32800,140625.0000,106499.187203
7,Non-sponsored,Open box,468900,255399,362149.5000,106750.500000


In [40]:
# Printing stats on condition and type 
cursor.execute("select item_condition,type, count(item_condition) from ebay.ebay_items group by type, item_condition;")
result = cursor.fetchall()
attr = []
for i in result:
    a = {}
    a = i
    attr.append(a)

In [41]:
attr = pd.DataFrame(attr)
attr.columns = ['Condition', 'Type', 'Count']
attr

,Condition,Type,Count
0,Used,Sponsored,45
1,Seller refurbished,Sponsored,2
2,New,Sponsored,71
3,Open box,Sponsored,2
4,New,Non-sponsored,14
5,Used,Non-sponsored,116
6,Seller refurbished,Non-sponsored,4
7,Open box,Non-sponsored,2


In [42]:
# Printing stats on availablity of best offer
cursor.execute("select count(best_offer), best_offer, type from ebay.ebay_items group by best_offer, type;")
result = cursor.fetchall()
avail = []
for i in result:
    a = {}
    a = i
    avail.append(a)

In [43]:
avail = pd.DataFrame(avail)
avail.columns = ['Count', 'Availability', 'Type']
avail

,Count,Availability,Type
0,100,Not Available,Sponsored
1,20,Available,Sponsored
2,84,Not Available,Non-sponsored
3,52,Available,Non-sponsored


In [44]:
# Printing stats on number of items sold 
cursor.execute("select type, item_condition, Sum(no_sold), max(no_sold) as max_sold, min(no_sold) as min_sold, avg(no_sold) as avg_sold, std(no_sold) as sold_sd from ebay.ebay_items group by type, item_condition;")
result = cursor.fetchall()
# Count Max Min Avg SD Type Condition
num = []
for i in result:
    n = {}
    n = i
    num.append(n)

In [45]:
num = pd.DataFrame(num)
num.columns = ['Type' , 'Condition', 'Sum', 'Max', 'Min', 'Avg', 'SD']
num

,Type,Condition,Sum,Max,Min,Avg,SD
0,Sponsored,Used,2,1,0,0.0444,0.206080
1,Sponsored,Seller refurbished,21,21,0,10.5000,10.500000
2,Sponsored,New,115,28,0,1.6197,4.001289
3,Sponsored,Open box,0,0,0,0.0000,0.000000
4,Non-sponsored,New,5664,2379,0,404.5714,710.489742
5,Non-sponsored,Used,4,3,0,0.0345,0.291578
6,Non-sponsored,Seller refurbished,3,3,0,0.7500,1.299038
7,Non-sponsored,Open box,0,0,0,0.0000,0.000000


In [46]:
# Printing stats on returns allowed
cursor.execute("select type, item_condition, sum(returns_allowed), max(returns_allowed), min(returns_allowed), avg(returns_allowed), std(returns_allowed) from ebay.ebay_items group by type, item_condition; ")
result = cursor.fetchall()
# Count Max Min Avg SD Type Condition
returns = []
for i in result:
    r = {}
    r = i
    returns.append(r)

In [47]:
returns = pd.DataFrame(returns)
returns.columns = ['Type', 'Condition', 'Sum', 'Max', 'Min', 'Avg', 'SD']
returns

,Type,Condition,Sum,Max,Min,Avg,SD
0,Sponsored,Used,41.0,1.0,0.0,0.911111,0.284583
1,Sponsored,Seller refurbished,2.0,1.0,1.0,1.000000,0.000000
2,Sponsored,New,68.0,1.0,0.0,0.957746,0.201167
3,Sponsored,Open box,1.0,1.0,0.0,0.500000,0.500000
4,Non-sponsored,New,13.0,1.0,0.0,0.928571,0.257539
5,Non-sponsored,Used,94.0,1.0,0.0,0.817391,0.386345
6,Non-sponsored,Seller refurbished,4.0,1.0,1.0,1.000000,0.000000
7,Non-sponsored,Open box,2.0,1.0,1.0,1.000000,0.000000


In [48]:
# Printing stats on shipping price
cursor.execute("select type, item_condition, max(shipping), min(shipping), avg(shipping), std(shipping) from ebay.ebay_items group by type, item_condition;")
result = cursor.fetchall()
# Max Min Avg SD Type Condition
shipping = []
for i in result:
    s = {}
    s = i
    shipping.append(i) 

In [49]:
shipping = pd.DataFrame(shipping)
shipping.columns = ['Type', 'Condition', 'Max', 'Mix', 'Avg', 'SD']
shipping

,Type,Condition,Max,Mix,Avg,SD
0,Sponsored,Used,7500.0,0.0,1603.244444,2173.221011
1,Sponsored,Seller refurbished,0.0,0.0,0.000000,0.000000
2,Sponsored,New,2000.0,0.0,28.169014,235.678881
3,Sponsored,Open box,15000.0,0.0,7500.000000,7500.000000
4,Non-sponsored,New,0.0,0.0,0.000000,0.000000
5,Non-sponsored,Used,3900.0,0.0,402.652174,834.177502
6,Non-sponsored,Seller refurbished,0.0,0.0,0.000000,0.000000
7,Non-sponsored,Open box,0.0,0.0,0.000000,0.000000


#### g

1. Looking at the price table we see that the max price of both new and old non-sponsored products is much greater. However their average prices are close if not similar. 
2. By looking at the attributes table we observe that there are many more non-sponsored products on Ebay than there are sponsored products. 
3. The avg and max seller score is higher for people selling new products, this makes sense as these might be registered vendors who have more customers who rate them. 
4. We also notice that the best offer option is more available for Non-sponsored posts
5. We also see that non-sponsored new products are sold the most followed by non-sponsored used products. 
6. Looking at the returns tables we see that non-sponsored new product have the most returns allowed which again follows intuition. 
7. The max shipping cost is $15000149.74 which makes no sense but ebay has some crazy people.
8. Since the best offer option is predominantly available for non-sponsored posts, you could use that to predict is a product is sponsored or non-sponsored. 